# Extract the dominant colors of skin, eyes and hair

In [4]:
import cv2
import numpy as np
import dlib
from collections import OrderedDict

In [10]:
FACIAL_LANDMARKS_IDXS = OrderedDict([
    #("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)), # hair
    #("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)), # eye
    #("left_eye", (42, 48)),
    ("nose", (27, 35)), # skin
    #("jaw", (0, 17))
])

def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

# Image is already a face no need to extract the face
def extract_face_parts(image, predictor_path):
    predictor = dlib.shape_predictor(predictor_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rect = dlib.rectangle(left=0, top=0, right=image.shape[1], bottom=image.shape[0])
    shape = predictor(gray, rect)
    shape = shape_to_np(shape)
    
    rois = []

    for (name, (i, j)) in FACIAL_LANDMARKS_IDXS.items():
        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        
        roi = image[y:y + h, x:x + w]
        roi = cv2.resize(roi, None, fx=5, fy=5)
        
        rois.append((x, y, w, h))
        
    
    return rois

def try_it(face_path):
    image = cv2.imread(face_path)
    extract_face_parts(image, "shape_predictor_68_face_landmarks.dat")

try_it("test_face.ppm")

(128, 128, 3)
